In [14]:
!pip install pycaret[full]

In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd

from transformers import RobertaTokenizer

from pycaret.classification import *

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/asrs-aeroguard/01_df_train_val_test.pkl


In [16]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [17]:
filepath = fr'/kaggle/input/asrs-aeroguard/01_df_train_val_test.pkl'
df_train_val_test = pd.read_pickle(filepath)

df_train = df_train_val_test.query('TRAIN_VAL_TEST_SPLIT == "Train"')
print(f'{df_train.shape=}')
df_val = df_train_val_test.query('TRAIN_VAL_TEST_SPLIT == "Validation"')
print(f'{df_val.shape=}')
df_test = df_train_val_test.query('TRAIN_VAL_TEST_SPLIT == "Test"')
print(f'{df_test.shape=}')

df_train

df_train.shape=(38655, 114)
df_val.shape=(4295, 114)
df_test.shape=(4773, 114)


,ACN_NUM_ACN,TIME_DATE,TIME_1_LOCAL_TIME_OF_DAY,PLACE_LOCALE_REFERENCE,PLACE_1_STATE_REFERENCE,PLACE_2_RELATIVE_POSITION_ANGLE_RADIAL,PLACE_3_RELATIVE_POSITION_DISTANCE_NAUTICAL_MILES,PLACE_4_ALTITUDE_AGL_SINGLE_VALUE,PLACE_5_ALTITUDE_MSL_SINGLE_VALUE,ENVIRONMENT_FLIGHT_CONDITIONS,...,ASSESSMENTS_CONTRIBUTING_FACTORS_SITUATIONS,ASSESSMENTS_1_PRIMARY_PROBLEM,REPORT_1_NARRATIVE,REPORT_1_1_CALLBACK,REPORT_2_NARRATIVE,REPORT_2_1_CALLBACK,REPORT_1_2_SYNOPSIS,TRAIN_VAL_TEST_SPLIT,EVENT_RISK,EVENT_RISK_STR
0,1574675,201808,0601-1200,SNA.Airport,CA,NaN,NaN,NaN,5000.0,NA,...,Human Factors,Human Factors,SNA RNP-Z to Runway 20R. The FMC was properly ...,NA,We were cleared for the RNP RNAV Z 20R Approac...,NA,B737-700 flight crew reported failing to make ...,Train,2,Medium risk
1,1224894,201412,0601-1200,MSY.Airport,LA,NaN,NaN,1000.0,NaN,VMC,...,Human Factors,Human Factors,On base to final turn to runway 1 in MSY at ap...,NA,NA,NA,Captain reports sighting of a drone at 1;000 f...,Train,0,Low risk
2,1134202,201312,1201-1800,ZZZ.ARTCC,US,NaN,NaN,NaN,2600.0,IMC,...,Human Factors; Aircraft; Procedure; Weather,Aircraft,I climbed to my filed altitude of 5;000 FT; an...,NA,NA,NA,SR22 pilot became disoriented on approach in I...,Train,3,Moderately high risk
3,1222074,201411,1201-1800,CWA.Airport,WI,NaN,20.0,NaN,4000.0,NA,...,Aircraft,Aircraft,I had my pitot heat checked prior to winter an...,NA,NA,NA,BE58 pilot experiences pitot heat failure desc...,Train,2,Medium risk
4,1733019,202003,1801-2400,ZDV.ARTCC,CO,NaN,NaN,NaN,32000.0,NA,...,Airspace Structure; Weather,Weather,At 32000 ft. just north of PUB the aircraft ex...,NA,NA,NA,B737 First Officer reported unexpected moderat...,Train,3,Moderately high risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38650,1093770,201306,NA,MRY.Airport,CA,330.0,3.0,NaN,2500.0,IMC,...,Human Factors; Aircraft,Ambiguous,Maintain directional control during climbout t...,NA,NA,NA,C172 instructor pilot reports difficulties wit...,Train,2,Medium risk
38651,1580993,201809,1201-1800,DTW.Airport,MI,NaN,NaN,250.0,NaN,NA,...,Environment - Non Weather Related; Procedure,Procedure,Given takeoff clearance from 21R in DTW behind...,NA,NA,NA,CRJ-900 Captain reported encountering wake tur...,Train,2,Medium risk
38652,1438881,201704,NA,ZZZ.Airport,US,NaN,NaN,0.0,NaN,NA,...,Company Policy; Human Factors,Company Policy,I did not witness passenger boarding as I was ...,NA,NA,NA,Boeing 787 Flight Attendant reported an adult ...,Train,0,Low risk
38653,1614309,201901,0601-1200,ZZZ.ARTCC,US,NaN,NaN,NaN,45000.0,VMC,...,Aircraft; Human Factors,Aircraft,Extreme cold cockpit temperatures were experie...,NA,[Report narrative contained no additional info...,NA,EMB-505 flight crew reported the loss of tempe...,Train,2,Medium risk


In [18]:
def preprocess_inputs(df):
    df = df.copy()
    
    X = df['REPORT_1_2_SYNOPSIS']
    y = df['EVENT_RISK']
    
    X = X.apply(lambda x: tokenizer.encode(x, padding='max_length', truncation=True))
    new_df = X.apply(pd.Series)
    new_df['label'] = y
    
    return new_df

In [19]:
df_pycaret = preprocess_inputs(df_train)
df_pycaret

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,label
0,0,387,406,3272,12,5987,2524,3419,431,4551,...,1,1,1,1,1,1,1,1,1,2
1,0,35645,690,30470,9,10,8678,23,112,131,...,1,1,1,1,1,1,1,1,1,0
2,0,17973,2036,4792,1059,2982,13283,15,1548,11,...,1,1,1,1,1,1,1,1,1,3
3,0,8827,4432,4792,3734,8516,1242,2859,2988,35602,...,1,1,1,1,1,1,1,1,1,2
4,0,387,406,3272,1234,2817,431,7152,7212,7,...,1,1,1,1,1,1,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38650,0,347,29250,14772,4792,690,9282,19,39,272,...,1,1,1,1,1,1,1,1,1,2
38651,0,9822,863,12,7784,8977,431,37476,3874,28742,...,1,1,1,1,1,1,1,1,1,2
38652,0,387,3540,154,262,5677,13275,7279,25113,431,...,1,1,1,1,1,1,1,1,1,0
38653,0,5330,387,12,35255,2524,3419,431,5,872,...,1,1,1,1,1,1,1,1,1,2


In [20]:
clf = setup(data=df_pycaret, target='label', session_id=42)

,Description,Value
0,Session id,42
1,Target,label
2,Target type,Multiclass
3,Original data shape,"(38655, 513)"
4,Transformed data shape,"(38655, 513)"
5,Transformed train set shape,"(27058, 513)"
6,Transformed test set shape,"(11597, 513)"
7,Numeric features,512
8,Preprocess,True
9,Imputation type,simple


In [21]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.4075,0.6583,0.4075,0.4020,0.3662,0.1677,0.1833,103.4140
lightgbm,Light Gradient Boosting Machine,0.4031,0.6544,0.4031,0.4067,0.3450,0.1458,0.1702,10.6310
catboost,CatBoost Classifier,0.3986,0.6510,0.3986,0.3887,0.3448,0.1461,0.1650,103.2780
gbc,Gradient Boosting Classifier,0.3948,0.6429,0.3948,0.3996,0.3108,0.1151,0.1514,47.8780
rf,Random Forest Classifier,0.3755,0.5998,0.3755,0.3679,0.2837,0.0840,0.1118,4.6130
ada,Ada Boost Classifier,0.3736,0.6008,0.3736,0.3360,0.2795,0.0805,0.1081,3.0590
et,Extra Trees Classifier,0.3728,0.6030,0.3728,0.3474,0.2825,0.0823,0.1078,7.0890
ridge,Ridge Classifier,0.3590,0.0000,0.3590,0.2996,0.2527,0.0535,0.0751,0.4710
lda,Linear Discriminant Analysis,0.3575,0.5719,0.3575,0.2943,0.2572,0.0555,0.0754,1.1920
lr,Logistic Regression,0.3569,0.5638,0.3569,0.2856,0.2581,0.0565,0.0751,16.2800


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [22]:
# Create the final model
final_model = create_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4072,0.6506,0.4072,0.4029,0.3624,0.1646,0.1814
1,0.4180,0.6573,0.4180,0.4215,0.3783,0.1817,0.1988
2,0.4150,0.6568,0.4150,0.4188,0.3773,0.1781,0.1952
3,0.4076,0.6602,0.4076,0.3993,0.3670,0.1698,0.1844
4,0.4106,0.6615,0.4106,0.3981,0.3699,0.1741,0.1889
5,0.4058,0.6615,0.4058,0.3949,0.3623,0.1643,0.1802
6,0.4024,0.6561,0.4024,0.3963,0.3613,0.1602,0.1756
7,0.4013,0.6667,0.4013,0.3985,0.3619,0.1589,0.1738
8,0.4089,0.6618,0.4089,0.4006,0.3625,0.1681,0.1845


Processing:   0%|          | 0/4 [00:00<?, ?it/s]